In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from scipy.ndimage import gaussian_filter1d

from scipy import stats
import statsmodels.formula.api as smf

sns.set_context('poster')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# import Python3_OpenOE_AC_map_functions_v1_08_30s as oem
import mz_LFP_functions as mz_LFP

#for the animation plot video at the very bottom!
import matplotlib.animation as ani

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    # optional, makes markers bigger, too, axes.linewidth doesn't seem to work
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
    'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
    #'axes.color_cycle':pal, # image.cmap - rewritesd the default colormap
    'axes.linewidth':2.5, 'lines.linewidth': 2.5,
    'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black','axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

#set_pub_bargraphs()
set_pub_plots()

In [ ]:
pal=sns.blend_palette(['gray','crimson'],2)
sns.palplot(pal)
sns.set_palette(pal)

# Load some necessary variables

In [ ]:
insert_depth = 3100  #change this as appropriate

sp_bw_ch = 20/2

surface_ch = np.round(insert_depth/sp_bw_ch)
V1_hip_ch = np.round((insert_depth-1100)/sp_bw_ch)
Hip_thal_ch = np.round((insert_depth-1100-1200)/sp_bw_ch)

CA1_DG_ch = np.round((insert_depth-1100-600)/sp_bw_ch)

print(surface_ch, V1_hip_ch, Hip_thal_ch, CA1_DG_ch)

In [5]:
samples_tr = 7350 #this is based on the shortest #samples in a trial
sr = 2500
n_chan = 384
rec_length = 3.0 #how long is the arduino triggered?

---

# First, load in the .npy arrays and CC_ls
These were creaded and saved using the "1_saving_LFP_arrays" jupyter notebook

In [6]:
all_pre_arr = np.load(r"D:\\pre_all_rew.npy")
all_post_arr = np.load(r"D:\\post_all_rew.npy")
all_novel_arr = np.load(r"D:\\novel_all_rew.npy")

In [ ]:
pkl_file = r"D:\\pre_et_ls"

open_file = open(pkl_file, "rb")
et_ls_pre = pickle.load(open_file)
open_file.close()

et_ls_pre

In [ ]:
pkl_file = r"D:\\post_et_ls"

open_file = open(pkl_file, "rb")
et_ls_post = pickle.load(open_file)
open_file.close()

et_ls_post

In [ ]:
pkl_file = r"D:\\novel_et_ls"

open_file = open(pkl_file, "rb")
et_ls_novel = pickle.load(open_file)
open_file.close()

et_ls_novel

---

# Plot the VEP ch trace

### First, we have to define a function
- dko: 
- h4ko: 
- h5ko: 
- wt: 

In [10]:
def VEP_lines(data_array, et_ls): #expand this to separate dko, h4ko, h5ko, and wt
    dko = []
    wt = []
    for i in range(data_array.shape[0]):
        if (et_ls[i]=="et305")|(et_ls[i]=="et3052")|(et_ls[i]=="et3053")|(et_ls[i]=="et320")|(et_ls[i]=="et3202")|(et_ls[i]=="et3203")|(et_ls[i]=="et379")|(et_ls[i]=="et3792")|(et_ls[i]=="et382")|(et_ls[i]=="et3822"):
            dko.append(data_array[i])
        else:
            wt.append(data_array[i])

    dko_arr = np.array(dko)
    wt_arr = np.array(wt)

    mean_dko = dko_arr.mean(axis=0)
    mean_wt = wt_arr.mean(axis=0)

    print('Group dko array: {0}'.format(dko_arr.shape))
    print('Group dko mean: {0}'.format(mean_dko.shape))

    print('Group wt array: {0}'.format(wt_arr.shape))
    print('Group wt mean: {0}'.format(mean_wt.shape))

    V1_dko = mean_dko[250:275, :]
    min_dko = np.where(V1_dko == np.amin(V1_dko))
    min_ch_dko = min_dko[0][0] + 249
    print(min_ch_dko)

    V1_wt = mean_wt[250:275, :]
    min_wt = np.where(V1_wt == np.amin(V1_wt))
    min_ch_wt = min_wt[0][0] + 249
    print(min_ch_wt)
    
    return mean_dko, mean_wt, min_ch_dko, min_ch_wt

### Second, apply the function to the three situations

In [ ]:
# Rewarded with water
pre_mean_dko, pre_mean_wt, pre_minch_dko, pre_minch_wt = VEP_lines(all_pre_arr, et_ls_pre)

In [ ]:
# Rewarded without water
post_mean_dko, post_mean_wt, post_minch_dko, post_minch_wt = VEP_lines(all_post_arr, et_ls_post)

In [ ]:
# Unrewarded
novel_mean_dko, novel_mean_wt, novel_minch_dko, novel_minch_wt = VEP_lines(all_novel_arr, et_ls_novel)

## Aside, can check individual mice if you want!

In [28]:
def plot_mouse_Vep(array, channel, group):
    plt.figure(figsize=(10, 5))
    if group == 'wt':
        plt_color = 'grey'
    elif group == 'dko':
        plt_color = 'crimson'
    mean_ch_traceA = array[channel,:]
    time_arr2_A = np.linspace(0, mean_ch_traceA.shape[0]/sr, mean_ch_traceA.shape[0])
    plt.plot(time_arr2_A, mean_ch_traceA, label='wt', color=plt_color)
    plt.xlabel('Time (s)')
    plt.ylabel('uV')
    plt.ylim([-400,300])
    plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='b')
    plt.show()
    
def VEP_lines_all_mice(data_array, et_ls):
    dko = []
    wt = []
    for i in range(data_array.shape[0]):
        if (et_ls[i]=="et305")|(et_ls[i]=="et3052")|(et_ls[i]=="et3053")|(et_ls[i]=="et320")|(et_ls[i]=="et3202")|(et_ls[i]=="et3203")|(et_ls[i]=="et379")|(et_ls[i]=="et3792")|(et_ls[i]=="et382")|(et_ls[i]=="et3822"):
            dko.append(data_array[i])
        else:
            wt.append(data_array[i])
    dko_arr = np.array(dko)
    wt_arr = np.array(wt)
    return dko_arr, wt_arr

def single_mouse_Veps(big_array, avg_ch, my_group):
    for mouse, m_arr in enumerate(big_array):
        print('Mouse index {}'.format(mouse))
        plot_mouse_Vep(m_arr, avg_ch, group=my_group)

In [20]:
# Rewarded with water
pre_all_dko, pre_all_wt = VEP_lines_all_mice(all_pre_arr, et_ls_pre)
# Rewarded without water
post_all_dko, post_all_wt = VEP_lines_all_mice(all_post_arr, et_ls_post)
# Unrewarded
novel_all_dko, novel_all_wt = VEP_lines_all_mice(all_novel_arr, et_ls_novel)

In [ ]:
# Pre Training individual mice
single_mouse_Veps(pre_all_dko, pre_minch_dko, my_group='dko')
single_mouse_Veps(pre_all_wt, pre_minch_wt, my_group='wt')

In [ ]:
# Post Training individual mice
single_mouse_Veps(post_all_dko, post_minch_dko, my_group='dko')
single_mouse_Veps(post_all_wt, post_minch_wt, my_group='wt')

In [ ]:
# Novel individual mice
single_mouse_Veps(novel_all_dko, novel_minch_dko, my_group='dko')
single_mouse_Veps(novel_all_wt, novel_minch_wt, my_group='wt')

### Finally, you can plot the graphs

In [ ]:
plt.figure(figsize=(10, 5))

mean_ch_traceA = pre_mean_wt[pre_minch_wt,:]
time_arr2_A = np.linspace(0, mean_ch_traceA.shape[0]/sr, mean_ch_traceA.shape[0])

mean_ch_traceB = pre_mean_dko[pre_minch_dko,:]
time_arr2_B = np.linspace(0, mean_ch_traceB.shape[0]/sr, mean_ch_traceB.shape[0])

plt.plot(time_arr2_A, mean_ch_traceA, label='wt', color='grey')
plt.plot(time_arr2_B, mean_ch_traceB, label='dko', color='crimson')
plt.legend(loc="lower right")

plt.title('Pre-training')
plt.xlabel('Time (s)')
plt.ylabel('uV')
plt.ylim([-400,300])
plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='b')

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\LFP\pre_VEP_trace.pdf", transparent=True) # fix this before running

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

mean_ch_traceA = post_mean_wt[post_minch_wt,:]
time_arr2_A = np.linspace(0, mean_ch_traceA.shape[0]/sr, mean_ch_traceA.shape[0])

mean_ch_traceB = post_mean_dko[post_minch_dko,:]
time_arr2_B = np.linspace(0, mean_ch_traceB.shape[0]/sr, mean_ch_traceB.shape[0])

plt.plot(time_arr2_A, mean_ch_traceA, label='wt', color='grey')
plt.plot(time_arr2_B, mean_ch_traceB, label='dko', color='crimson')
plt.legend(loc="lower right")

plt.title('Post-training')
plt.xlabel('Time (s)')
plt.ylabel('uV')
plt.ylim([-400,300])
plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='b')

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\LFP\post_VEP_trace.pdf", transparent=True) # fix this before running

plt.show()

In [ ]:
plt.figure(figsize=(10, 5))

mean_ch_traceA = novel_mean_wt[novel_minch_wt,:]
time_arr2_A = np.linspace(0, mean_ch_traceA.shape[0]/sr, mean_ch_traceA.shape[0])

mean_ch_traceB = novel_mean_dko[novel_minch_dko,:]
time_arr2_B = np.linspace(0, mean_ch_traceB.shape[0]/sr, mean_ch_traceB.shape[0])

plt.plot(time_arr2_A, mean_ch_traceA, label='wt', color='grey')
plt.plot(time_arr2_B, mean_ch_traceB, label='dko', color='crimson')
plt.legend(loc="lower right")

plt.title('Novel')
plt.xlabel('Time (s)')
plt.ylabel('uV')
plt.ylim([-400,300])
plt.axvspan(0.5, 0.7, alpha=0.2, facecolor='b')

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\LFP\novel_VEP_trace.pdf", transparent=True) # fix this before running

plt.show()

---

# Finding the VEP peaks in each situation


In [46]:
time_win=[[0.55,0.75],   # VEP 1 time frame
          [0.76,0.85],  # VEP 2 time frame
          [0.86,1],     # VEP 3 time frame
          [1.05,1.15],  # VEP 4 time frame
          [1.2,1.4]]    # VEP 5 time frame


time_win=[[i[0]*sr,i[1]*sr] for i in time_win]
depth_b=[[200,300],[0,200]] #V1 bounds and hippocampus/Thalamus bounds


In [ ]:
pre_V1vepdf = mz_LFP.create_vepamp_df(all_pre_arr, time_windows=time_win, depth_bounds=depth_b)
post_V1vepdf = mz_LFP.create_vepamp_df(all_post_arr, time_windows=time_win, depth_bounds=depth_b)
novel_V1vepdf = mz_LFP.create_vepamp_df(all_novel_arr, time_windows=time_win, depth_bounds=depth_b)

# r1_V1vepdf.head()

### Second, we add in some additonal info to the dfs

In [48]:
et_ls = et_ls_pre

et_dfls = []
groupls = []
for i in pre_V1vepdf.rec_ind:
    et_dfls.append(et_ls[i])
    if (et_ls[i]=="et382")|(et_ls[i]=="et376")|(et_ls[i]=="et379")|(et_ls[i]=="et3792")|(et_ls[i]=="et305")|(et_ls[i]=="et326")|(et_ls[i]=="et320")|(et_ls[i]=="et3202"):
        groupls.append("dko")
    else:
        groupls.append("wt")

pre_V1vepdf["et"] = et_dfls
pre_V1vepdf["group"] = groupls

In [49]:
et_ls = et_ls_post

et_dfls = []
groupls = []
for i in post_V1vepdf.rec_ind:
    et_dfls.append(et_ls[i])
    if (et_ls[i]=="et382")|(et_ls[i]=="et376")|(et_ls[i]=="et379")|(et_ls[i]=="et3792")|(et_ls[i]=="et305")|(et_ls[i]=="et326")|(et_ls[i]=="et320")|(et_ls[i]=="et3202"):
        groupls.append("dko")
    else:
        groupls.append("wt")

post_V1vepdf["et"] = et_dfls
post_V1vepdf["group"] = groupls

In [50]:
et_ls = et_ls_novel

et_dfls = []
groupls = []
for i in novel_V1vepdf.rec_ind:
    et_dfls.append(et_ls[i])
    if (et_ls[i]=="et382")|(et_ls[i]=="et376")|(et_ls[i]=="et379")|(et_ls[i]=="et3792")|(et_ls[i]=="et305")|(et_ls[i]=="et326")|(et_ls[i]=="et320")|(et_ls[i]=="et3202"):
        groupls.append("dko")
    else:
        groupls.append("wt")

novel_V1vepdf["et"] = et_dfls
novel_V1vepdf["group"] = groupls

In [ ]:
pre_V1vepdf.head()

### Finally, we can plot the quantification results

In [ ]:
sns.catplot(data=pre_V1vepdf,
            x='vep',
            y='vepamp',
            kind='point',
            hue='group',
            ci=68,
            legend=False,
            hue_order=['wt','dko'],
            height = 4.5,
            aspect=1.5,
            palette=sns.color_palette(['grey', 'crimson'])
           )

# plt.legend(loc="lower right")
plt.ylabel('uV')
plt.xlabel('Oscillation Cycle')
plt.title('Pre training')
plt.ylim([0,500])
plt.yticks([0,250,500])

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\\pre_VEP_quant.pdf", transparent=True)

plt.show()

---

# Testing the stats about the above plots
### Overall test for the genotypes (wt vs dko)

In [ ]:
pre_stats, pre_fstats = mz_LFP.VEP_overall_stats(pre_V1vepdf, group_ls=['wt','dko'])
post_stats, post_fstats = mz_LFP.VEP_overall_stats(post_V1vepdf, group_ls=['wt','dko'])
novel_stats, novel_fstats = mz_LFP.VEP_overall_stats(novel_V1vepdf, group_ls=['wt','dko'])

In [ ]:
print('~~~~~ Pre training ~~~~~')
print(pre_stats)
print(pre_fstats)

In [ ]:
print('~~~~~ Post training ~~~~~')
print(post_stats)
print(post_fstats)

In [ ]:
print('~~~~~ Novel ~~~~~')
print(novel_stats)
print(novel_fstats)

### Individual tests for each vep peak

In [91]:
prepeak_stats, prepeak_fstats = mz_LFP.VEP_peaks_stats(pre_V1vepdf, group_ls=['wt','dko'])
postpeak_stats, postpeak_fstats = mz_LFP.VEP_peaks_stats(post_V1vepdf, group_ls=['wt','dko'])
novelpeak_stats, novelpeak_fstats = mz_LFP.VEP_peaks_stats(novel_V1vepdf, group_ls=['wt','dko'])

In [ ]:
print('~~~~~ Pre training ~~~~~')
print(prepeak_stats)
print(prepeak_fstats)

In [ ]:
print('~~~~~ Post training ~~~~~')
print(postpeak_stats)
print(postpeak_fstats)

In [ ]:
print('~~~~~ Novel ~~~~~')
print(novelpeak_stats)
print(novelpeak_fstats)

### Linear Mixed Model analysis
First, for the Amp values x Vep # with a comparison between groups

Second, for the Amp values x group with a comparison between Vep #

In [ ]:
pre_mdf1, pre_mdf2 = mz_LFP.LMM_stats(pre_V1vepdf)
print(pre_mdf1.pvalues)
print(pre_mdf2.pvalues)

In [ ]:
post_mdf1, post_mdf2 = mz_LFP.LMM_stats(post_V1vepdf)
print(post_mdf1.pvalues)
print(post_mdf2.pvalues)

In [ ]:
novel_mdf1, novel_mdf2 = mz_LFP.LMM_stats(novel_V1vepdf)
print(novel_mdf1.pvalues)
print(novel_mdf2.pvalues)

---

---